Из визуализации мы заметили, что если %K и MACD оба имеют высокие/низкие значения, то изменение цены будет положительное/отрицательное. Тогда мы можем построить модель, которая при высоких значениях показателей будет прогнозировать рост цены, а при низких - её падение (в иных случаях на основе этих данных прогнозировать движение цены сложно) 

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [19]:
df = pd.read_csv('data.csv')

In [20]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

In [21]:
df.drop(columns=['Low_n','High_n','EMA_short', 'EMA_long','UpperBand_5h', 'LowerBand_5h', 'date','high_low_diff_15m',
       'high_low_diff_1h'], inplace=True)

In [22]:
def predict(df: pd.DataFrame):
  predictions = np.empty(df.shape[0], dtype=object)
  threshold = 0.5
  K = df['%K']
  MACD = df['MACD_Histogram']
  for i in range(df.shape[0]):
    if K[i] > 80 and MACD[i] > threshold:
      predictions[i] = 1
    if K[i] < 20 and MACD[i] < -threshold: 
      predictions[i] = -1
    else:
      predictions[i] = 0
  return predictions

In [23]:
def accuracy_score(y_true, y_pred) -> float:
    y_true = np.asarray(y_true).reshape(-1, 1)
    y_pred = np.asarray(y_pred).reshape(-1, 1)
    return (y_true == y_pred).mean()

In [25]:
df['direction'] = np.where(df['target_change_15m_pct'] > 0, 1, -1)

predictions = predict(df)
df['predictions'] = predictions

filtered_df = df[(df['predictions'] != 0)]
accuracy_filtered = accuracy_score(filtered_df['direction'], filtered_df['predictions'])

print(f"Accuracy for growth/fall predictions: {accuracy_filtered:.2f}")

Accuracy for growth/fall predictions: 0.86
